In [34]:
import os # saving files
import numpy as np # for matrix math
import matplotlib.pyplot as plt # plot and visualizing data
from data_utils import shuffle, iter_data # analyzing data
from tqdm import tqdm # progress bar
import preprocessing
import post_processing

In [35]:
#读取数据
NiH = preprocessing.read_data('data/NiH_dataset.mat')
PdH = preprocessing.read_data('data/PdH_dataset.mat')
print(np.shape(NiH))
print(np.shape(PdH))

(35, 4)
(35, 4)


In [36]:
#获取每种元素的最大行数
max_PdH = preprocessing.compute_max(PdH)
max_NiH = preprocessing.compute_max(NiH)
max_len=max(max_PdH,max_NiH)
max_len

18

In [37]:
#做数据填充（0）
Ni_H = preprocessing.data_padding(NiH,max_len)
Pd_H = preprocessing.data_padding(PdH,max_len)
print(np.array(Ni_H).shape)
print(np.array(Pd_H).shape)

(35, 4, 18, 3)
(35, 4, 18, 3)


In [38]:
#定义超参数
n_epoch =100
batch_size  = 35
input_dim = 3
latent_dim = 3
eps_dim = 3

# 判别器
n_layer_disc = 5
n_hidden_disc = 100

# 生成器
n_layer_gen = 5
n_hidden_gen= 100

# 另一个生成器
n_layer_inf = 5
n_hidden_inf= 100

In [39]:
#创建结果文件夹
result_dir = 'STEP1+STEP2_CrsytalGAN/'
directory = result_dir
if not os.path.exists(directory):
    os.makedirs(directory)
# AH和BH的真实样本
# 在本例中是Pd_H和Ni_H
AH_dataset = Pd_H
BH_dataset = Ni_H
print(np.array(AH_dataset).shape)
print(np.array(BH_dataset).shape)

(35, 4, 18, 3)
(35, 4, 18, 3)


In [40]:
from Dataset import AHBDataset
from torch.utils.data import DataLoader
dataSet=AHBDataset(AH_dataset,BH_dataset)
dataLoader=DataLoader(dataSet,batch_size=batch_size,shuffle=True)

In [46]:
print(batch_size)
for X,y in dataLoader:
    print(X)

    print('*'*40)

35
[tensor([[[ 2.6677e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  2.6677e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  2.6677e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 4.1298e+00, -2.0000e-16,  0.0000e+00],
         [ 2.0000e-16,  4.1152e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  4.5060e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

        [[ 4.0494e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  4.0440e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  4.2056e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

        ...,

        [[ 3.9562e+00,  0.0000e+00